# Time Series Forcasting Model based on LSTM with 'sunspots' dataset - Clone Project([Time Series-Tensorflow RNN-LSTM Introduction](https://www.kaggle.com/kutaykutlu/time-series-tensorflow-rnn-lstm-introduction))

## Step 0. Libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
print(tf.__version__)

import warnings
warnings.filterwarnings('ignore')

/Users/swsong/opt/anaconda3/envs/dataScience/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
2.2.0


## Step 1. The Functions
- Let's built some useful function to use in the next steps 

In [2]:
# It is a simple function to plot time x axis values y axis

def plot_series(time, series, format='-', start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.grid(True)

In [3]:
# Inputs: data series, size of the window, The size of the batches to use when training, The size of the shuffle buffer, which determines how the data will be shuffled.
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # Expand the dimensions of the series to work with the LSTM's
    series = tf.expand_dims(series, axis=-1)
    # Create dataset ds from the series
    ds = tf.data.Dataset.from_tensor_slices(series)
    # Slice the data up into the appropriate windows, shifted by one time set.
    # Keep them all the same size by setting drop remainder to true.
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    # Flatten the data into chunks in the size of our window_size+1
    ds = ds.flat_map(lambda w : w.batch(window_size+1))
    # Shuffle it with shuffle buffer that speeds things up with large datasets
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w : (w[:-1], w[1:]))
    # Return the dataset that batched into the selected batch size
    return ds.batch(batch_size).prefetch(1)
    

In [5]:
# We can use this function to make some prediction by using the trained mode 1
def model_forcast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w : w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.prediction(ds)
    return forecast